# DEBUGGING TIMESKETCH API WITH PYTHON CLI

This notebook is intended to by useful when debugging Timesketch: it can create/delete/visualize sketches and timelines, retrieve analyzers' results, run queries on a sketch and many others. Feel free to use and modify the section that you find more useful.

In [ ]:
!pip install -q timesketch_api_client

In [ ]:
import json

from timesketch_api_client import config
from timesketch_api_client import sketch
from timesketch_api_client import timeline
from timesketch_api_client import analyzer
from timesketch_api_client import search

from timesketch_import_client import helper
from timesketch_import_client import importer

## Initialization section
Define here the name of the sketch

In [ ]:
sketch_name = "0p3nS0C"

In [ ]:
ts_client = config.get_client()

## GET section
This section has the command to retrieve the list of sketches, timelines, analyzers...

### GET list of sketches

In [ ]:
sketches = [(x.id, x.name) for x in ts_client.list_sketches()]
for sketch in sketches:
    print(f"Sketch name: {sketch[1]} ({sketch[0]})")

### GET list of sketch - timeline - datasource

In [ ]:
sketches = [(x.id, x.name) for x in ts_client.list_sketches()]
for sketch_info in sketches:
    try:
        sketch = ts_client.get_sketch(sketch_info[0])
        print(f"[S] {sketch.name} ({sketch.id})")
        for timeline in sketch.list_timelines():
            print(f"   - [T] {timeline.name} ({timeline.id})")
            for datasource in timeline.data_sources:
                print(f"      - [D] {datasource['context']} ({datasource['status'][0]['status']})")
    except Exception as e:
        print(f"[S]{sketch_info[1]} ({sketch_info[0]}) skipped")

### GET detailed events of a sketch

In [ ]:
sketch_ID = 1
print(f"Using sketch ID {sketch_ID}")
sketch = ts_client.get_sketch(sketch_ID)
print(json.dumps(sketch.data, indent=4))

### GET detailed events of a timeline

In [ ]:
import json
timeline_ID = 12

# first need to find the sketch of that timeline
sketches = [(x.id, x.name) for x in ts_client.list_sketches()]
for sketch_info in sketches:
    try:
        sketch = ts_client.get_sketch(sketch_info[0])
        timelines_ID = [x.id for x in sketch.list_timelines()]
        if timeline_ID in timelines_ID:
            print(f"Found timeline ID {timeline_ID} in sketch {sketch_info[1]}")
            print(f"Print information of timeline ID {timeline_ID}...")
            timeline_info = sketch.get_timeline(timeline_ID).data
            print(json.dumps(timeline_info, indent=4))
            break
    except Exception as e:
        print(f"Cannot retrieve timeline ID {timeline_id}")

### GET events by running a QUERY

In [ ]:
sketch_ID = 1
screenshot_events = None

try:
    sketch = ts_client.get_sketch(sketch_ID)
    query = ('*')

    search_obj = search.Search(sketch)

    search_obj.query_string = query
    screenshot_events = search_obj.table
except Exception as e:
    print(f"Cannot find sketch ID {sketch_ID}")

print(f"Query: {query}")
screenshot_events

## CREATE section
In this section the are commands to create new sketch, a single timeline, a single timeline with multiple datasource and multiple timelines.

### Create a new sketch (if the name is unique)

In [ ]:
sketches = [(x.id, x.name) for x in ts_client.list_sketches() if x.name == sketch_name]
sketch = None
if sketches:
    print("Sketch already exists")
    sketch = ts_client.get_sketch(sketches[0][0])
else:
    print("Create new sketch")
    sketch = ts_client.create_sketch(name=sketch_name)

sketch_ID = sketch.id
print(f"Sketch information: {sketch.name} ({sketch_ID})")

### Create new timeline given a sketch

In [ ]:
filename = "/tmp/sample_chrome_logs.plaso"
timeline_name = "_".join(filename.split("/")[-1].split(".")[:-1])

sketches = [(x.id, x.name) for x in ts_client.list_sketches() if x.name == sketch_name]
sketch = None
if sketches:
    print(f"Using sketch {sketch_name}")
    sketch = ts_client.get_sketch(sketches[0][0])
    import_helper = helper.ImportHelper() 
    with importer.ImportStreamer() as streamer:
        streamer.set_sketch(sketch)
        streamer.set_config_helper(import_helper)
        streamer.set_timeline_name(timeline_name)
        streamer.add_file(filename)
        print(f"Timeline {timeline_name} inserted")
else:
    print("No sketch with that name")

### Create timelines given a list of files

In [ ]:
# feel free to modify these lines as they are only used to fill the array list_files
import os
dir_path = r'/tmp/list_files'

# list to store files
list_files = []

# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a file
    if os.path.isfile(os.path.join(dir_path, path)):
        list_files.append(dir_path + '/' + path)
print(f"List of files: {', '.join(list_files)}")

In [ ]:
sketches = [(x.id, x.name) for x in ts_client.list_sketches() if x.name == sketch_name]
sketch = None
if sketches:
    print(f"Using sketch {sketch_name}")
    sketch = ts_client.get_sketch(sketches[0][0])
    import_helper = helper.ImportHelper()     
    for filename in list_files:
        timeline_name = "_".join(filename.split("/")[-1].split(".")[:-1])
        try:
            with importer.ImportStreamer() as streamer:
                streamer.set_sketch(sketch)
                streamer.set_config_helper(import_helper)
                streamer.set_timeline_name(timeline_name)
                streamer.add_file(filename)
                print(f"Timeline {timeline_name} inserted")
        except Exception as e:
            print(f"{filename} skipped")
else:
    print(f"no sketch found with name {sketch_name}")

### Create a manual event

In [ ]:
datetime = "2020-08-06T12:48:06.994188Z"
message = "only a test to insert a manual event v2"
timestamp_desc = "creation time"
attributes = {"a": "alpha", "o": "omega", "g": "gamma"}
tags = ["this", "field", "is", "optional",]
sketches = [(x.id, x.name) for x in ts_client.list_sketches() if x.name == sketch_name]
sketch = None
if sketches:
    print(f"Using sketch {sketch_name}")
    sketch = ts_client.get_sketch(sketches[0][0])
    sketch.add_event(message, datetime, timestamp_desc, attributes, tags)
    print("Manual event added")
else:
    print(f"no sketch found with name {sketch_name}")    

## DELETE section

### Delete one or more timelines in the sketch

In [ ]:
timeline_names = ["ac01-chrome-history"]
sketches = [(x.id, x.name) for x in ts_client.list_sketches() if x.name == sketch_name]
sketch = None
if sketches:
    print(f"Using sketch {sketch_name}")
    sketch = ts_client.get_sketch(sketches[0][0])
    for timeline_name in timeline_names:
        timeline = sketch.get_timeline(timeline_name=timeline_name)
        if timeline:
            timeline.delete()
            print(f"Timeline {timeline_name} deleted")
        else:
            print(f"Timeline {timeline_name} not found")
else:
    print(f"no sketch found with name {sketch_name}")

### Delete all the timelines in the sketch

In [ ]:
sketches = [(x.id, x.name) for x in ts_client.list_sketches() if x.name == sketch_name]
sketch = None
if sketches:
    print(f"Using sketch {sketch_name}")
    sketch = ts_client.get_sketch(sketches[0][0])
    timelines = sketch.list_timelines()
    for timeline in timelines:
        try:
            timeline_name = timeline.name
            timeline.delete()
            print(f"Timeline {timeline_name} deleted")
        except Exception as e:
            print(f"ERROR: Timeline {timeline_name} skipped")
else:
    print(f"no sketch found with name {sketch_name}")

## ANALYZER Section
In this section the user can list and run the analyzers available on Timesketch

### GET list of available analyzers

In [ ]:
sketch_ID = 1
print(f"Listing analyzers available for sketch ID {sketch_ID}")
for list_analyzers in ts_client.get_sketch(sketch_ID).list_available_analyzers():
    print(json.dumps(list_analyzers, indent=4))

### Retrieve the result of an analyzer
Retrieve the results of an analyzer given a timeline ID

In [ ]:
analyzer_name = "domain"
timeline_ID = 12
sketches = [(x.id, x.name) for x in ts_client.list_sketches()]
for sketch_info in sketches:
    try:
        sketch = ts_client.get_sketch(sketch_info[0])
        timelines_ID = [x.id for x in sketch.list_timelines()]
        if timeline_ID in timelines_ID:
            print(f"Found timeline ID {timeline_ID} in sketch {sketch_info[1]}")
            print(f"Run anylyzer {analyzer_name} for timeline ID {timeline_ID}...")
            timeline = sketch.get_timeline(timeline_ID)
            analyzer = timeline.run_analyzer(analyzer_name=analyzer_name, ignore_previous=True)
            print("Run below cells to get the analyzer's result")
            break
    except Exception as e:
        print(f"Cannot run analyzer on timeline ID {timeline_ID}")

In [ ]:
all_results = [x._fetch_data() for x in analyzer]
all_results

## AGGREGATION Section
In this section the user can run aggregation query and more ... 